# Machine Learning avec Spark MLLib

Dans ce notebook, nous allons faire une démonstration de Spark MLLib sur des données d'exemple. Pour cela, nous allons appliquer les étapes d'un projet de machine learning :

- Création d'un jeu de données d'entraînement et de test
- Création de modèles
- Optimisation des hyperparamètres des modèles
- Evaluation des modèles

In [1]:
// notebook setup
print("Hello, World from Scala kernel!")

Intitializing Scala interpreter ...

Spark Web UI available at http://3c99054b149a:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1704568349298)
SparkSession available as 'spark'


Hello, World from Scala kernel!

# Création d'un jeu de données d'entraînement et de test

Dans un premier temps, chargeons le jeu de données.

In [2]:
val data = spark.read.format("libsvm").load("../spark/data/mllib/sample_libsvm_data.txt")

data: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [3]:
data.toDF().show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



Nous pouvons désormais créer un jeu d'entraînement et de test, en divisant le jeu de données aléatoirement en deux parties.

Le jeu d'entraînement contiendra 80% des données, et le jeu de test 20%.

Le jeu d'entraînement sera utilisé pour entraîner le modèle, et le jeu de test pour l'évaluer.

In [4]:
val trainingSet = data.sample(false, 0.8)
val testSet = data.except(trainingSet)

trainingSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


# Création de modèles

Le jeu de données représentant un problème de classification, nous allons utiliser des modèles de classification :
- la régression logistique
- les arbres de décision

In [10]:
import org.apache.spark.ml.classification._;
import org.apache.spark.ml.evaluation._;
import org.apache.spark.ml.tuning._;

import org.apache.spark.ml.classification._
import org.apache.spark.ml.evaluation._
import org.apache.spark.ml.tuning._


Nous allons créer des modèles, et les entraîner sur le jeu d'entraînement.

In [16]:
val decisionTree = new DecisionTreeClassifier().fit(trainingSet)
val logisticRegression = new LogisticRegression().fit(trainingSet)

decisionTree: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_7848116ac7f1, depth=2, numNodes=5, numClasses=2, numFeatures=692
logisticRegression: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_392d9925285b, numClasses=2, numFeatures=692
randomForest: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel: uid=rfc_2a16572cb413, numTrees=20, numClasses=2, numFeatures=692


# Optimisation des hyperparamètres du modèle

Une façon d'améliorer les performances d'un modèle est d'optimiser ses hyperparamètres à l'aide d'une recherche par grille.

Pour rappel, les hyperparamètres sont des paramètres du modèle qui ne sont pas appris lors de l'entraînement, mais qui sont fixés avant l'entraînement. Par exemple, le nombre d'arbres dans une forêt aléatoire est un hyperparamètre.

La recherche par grille consiste quant à elle à tester un ensemble de combinaisons d'hyperparamètres possibles, et à sélectionner celle qui donne les meilleurs résultats. Nous pouvons utiliser l'objet `ParamGridBuilder` de Spark pour cela.

L'optimisation des hyperparamètres est généralement effectuées sur un jeu de validation. Nous pouvons utiliser l'objet `TrainValidationSplit` de Spark pour cela.

## Régression logistique

Des hyperparamètres possibles de la régression logistique sont :
- `maxIter` : le nombre d'itérations
- `regParam` : le paramètre de régularisation

In [14]:
val logisticRegressionParameterGrid = new ParamGridBuilder()
  .addGrid(logisticRegression.maxIter, Array(10, 100, 1000))
  .addGrid(logisticRegression.regParam, Array(0.1, 0.01, 0.001))
  .build() 

val logisticRegressionTrainValidationSplit = new TrainValidationSplit()
  .setEstimator(new LogisticRegression())
  .setEvaluator(new MulticlassClassificationEvaluator)
  .setEstimatorParamMaps(logisticRegressionParameterGrid)
  .setTrainRatio(0.8)

logisticRegressionParameterGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_61b7158a6c73-maxIter: 10,
	logreg_61b7158a6c73-regParam: 0.1
}, {
	logreg_61b7158a6c73-maxIter: 100,
	logreg_61b7158a6c73-regParam: 0.1
}, {
	logreg_61b7158a6c73-maxIter: 1000,
	logreg_61b7158a6c73-regParam: 0.1
}, {
	logreg_61b7158a6c73-maxIter: 10,
	logreg_61b7158a6c73-regParam: 0.01
}, {
	logreg_61b7158a6c73-maxIter: 100,
	logreg_61b7158a6c73-regParam: 0.01
}, {
	logreg_61b7158a6c73-maxIter: 1000,
	logreg_61b7158a6c73-regParam: 0.01
}, {
	logreg_61b7158a6c73-maxIter: 10,
	logreg_61b7158a6c73-regParam: 0.001
}, {
	logreg_61b7158a6c73-maxIter: 100,
	logreg_61b7158a6c73-regParam: 0.001
}, {
	logreg_61b7158a6c73-maxIter: 1000,
	logreg_61b7158a6c73-regParam: 0.001
})
logisticRegressionTrainValidat...


In [17]:
val logisticRegression = logisticRegressionTrainValidationSplit.fit(trainingSet)

logisticRegression: org.apache.spark.ml.tuning.TrainValidationSplitModel = TrainValidationSplitModel: uid=tvs_efceeaaa5e65, bestModel=LogisticRegressionModel: uid=logreg_5b455abad2a0, numClasses=2, numFeatures=692, trainRatio=0.8


## Arbres de décision

Des hyperparamètres possibles des arbres de décision sont :
- `impurity` : la mesure d'impureté (indice de Gini ou l'entropie)
- `maxDepth` : la profondeur maximale de l'arbre
- `minInstancesPerNode` : le nombre minimal d'instances par noeud dasn l'arbre

In [23]:
val decisionTreeParameterGrid = new ParamGridBuilder()
  .addGrid(decisionTree.maxDepth, Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12))
  .addGrid(decisionTree.minInstancesPerNode, Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
  .addGrid(decisionTree.impurity, Array("gini", "entropy"))
  .build()

val decisionTreeTrainValidationSplit = new TrainValidationSplit()
  .setEstimator(new DecisionTreeClassifier())
  .setEvaluator(new MulticlassClassificationEvaluator)
  .setEstimatorParamMaps(decisionTreeParameterGrid)
  .setTrainRatio(0.8)

decisionTreeParameterGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_7848116ac7f1-impurity: gini,
	dtc_7848116ac7f1-maxDepth: 1,
	dtc_7848116ac7f1-minInstancesPerNode: 1
}, {
	dtc_7848116ac7f1-impurity: entropy,
	dtc_7848116ac7f1-maxDepth: 1,
	dtc_7848116ac7f1-minInstancesPerNode: 1
}, {
	dtc_7848116ac7f1-impurity: gini,
	dtc_7848116ac7f1-maxDepth: 2,
	dtc_7848116ac7f1-minInstancesPerNode: 1
}, {
	dtc_7848116ac7f1-impurity: entropy,
	dtc_7848116ac7f1-maxDepth: 2,
	dtc_7848116ac7f1-minInstancesPerNode: 1
}, {
	dtc_7848116ac7f1-impurity: gini,
	dtc_7848116ac7f1-maxDepth: 3,
	dtc_7848116ac7f1-minInstancesPerNode: 1
}, {
	dtc_7848116ac7f1-impurity: entropy,
	dtc_7848116ac7f1-maxDepth: 3,
	dtc_7848116ac7f1-minInstancesPerNode: 1
}, {
	dtc_7848116ac7f1-impurity: gini,
	dtc_7848...


In [24]:
val decisionTree = decisionTreeTrainValidationSplit.fit(trainingSet)

decisionTree: org.apache.spark.ml.tuning.TrainValidationSplitModel = TrainValidationSplitModel: uid=tvs_943a0b55ffaa, bestModel=DecisionTreeClassificationModel: uid=dtc_74c79fb7f64e, depth=2, numNodes=5, numClasses=2, numFeatures=692, trainRatio=0.8


## Evaluation des modèles

Nous pouvons désormais évaluer les modèles sur le jeu de test.

Pour cela, nous allons utiliser la métrique d'exactitude (accuracy).

In [26]:
val logisticRegressionPredictions = logisticRegression.transform(testSet)
val decisionTreePredictions = decisionTree.transform(testSet)

val logisticRegressionAccuracy = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
  .evaluate(logisticRegressionPredictions)

val decisionTreeAccuracy = new MulticlassClassificationEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("accuracy")
    .evaluate(decisionTreePredictions)

println("Logistic Regression Accuracy: " + logisticRegressionAccuracy)
println("Decision Tree Accuracy: " + decisionTreeAccuracy)

Logistic Regression Accuracy: 1.0
Decision Tree Accuracy: 1.0


logisticRegressionPredictions: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 3 more fields]
decisionTreePredictions: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 3 more fields]
logisticRegressionAccuracy: Double = 1.0
decisionTreeAccuracy: Double = 1.0


La régression logistique et l'arbre de décision ont une accuracy de 100% sur le jeu de test : ils prédisent correctement la classe de toutes les instances du jeu de test !